# Part 1: Horse Racing Prediction

 ## Data Preprocessing

In [ ]:
!pip install ipython-autotime

In [ ]:
from numba import jit, cuda
import pandas as pd
import numpy as np

# rcParams
# change image to white background
import matplotlib as mpl
mpl.rcParams['figure.facecolor'] = 'white'

%load_ext autotime

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Read the data
rrh = pd.read_csv('D:\\documentos\\IA Caballos\\hurdle_data\\Hurdle_race_course.csv')
rrr = pd.read_csv('D:\\documentos\\IA Caballos\\hurdle_data\\Hurdle_race.csv')
#rrr1 = pd.read_csv('C:\\Users\ROOM\Documents\\IA Caballos\\data\\records.csv')
print(rrh.shape)
print(rrr.shape)

In [ ]:
#rrr1.head(2)
rrh["jockey_hurdle_rate"] = rrh["jockey_hurdle_rate"].str.rstrip('%').astype('float')
#rrh["jockey_flat_turf_rate"] = rrh["jockey_flat_turf_rate"].str.rstrip('%').astype('float')
rrh["trainers_hurdle_rate"] = rrh["trainers_hurdle_rate"].str.rstrip('%').astype('float')
#rrh["trainers_flat_turf_rate"] = rrh["trainers_flat_turf_rate"].str.rstrip('%').astype('float')



In [ ]:
rrh.fillna(0, inplace=True)

In [ ]:
rrh.horse_rate = rrh.horse_rate.replace({"-": 0})

In [ ]:
import re
from numba import jit, cuda

@jit(target_backend='cuda')
def convert_to_km(distance):
    '''
    distance can be a string with km or m as units
    e.g. 300km, 1.1km, 200m, 4.5m
    '''
    
    # split the string into value and unit ['300', 'km']
    #split_dist = re.match('([\d\.]+)?([a-zA-Z]+)', distance)
    split_dist = re.findall('([\d\.]+)?([a-zA-Z]+)', distance, re.U)
    print(split_dist)
    dist = 0.0
    dist1 = 0.0
    dist2 = 0.0
    dist3 = 0.0
    for value in split_dist:
        if value[1] == 'm':
            dist1 = float(value[0])
        elif value[1] == 'f':
            dist2 = float(value[0])*0.125
        elif value[1] == 'y':
            dist3 = float(value[0])*0.0005681818
        else:
            pass
      
    dist = dist1 + dist2 + dist3
    
    return dist

In [ ]:
rrh['distance'] = rrh.apply(lambda row: convert_to_km(row['distance']), axis=1)

In [ ]:
#'race_distance'
rrr['Distance'] = rrr.apply(lambda row: convert_to_km(row['Distance']), axis=1)

In [ ]:
# View the data
rrh.head(2)

In [ ]:
# View the data
rrr.head(2)

In [ ]:
# Explore the data
rrh.info()

In [ ]:
# Explore the data
rrr.info()

## Data Cleaning

In [ ]:
# Look at the unique values in the column 'finishing_position'
rrh['finishing_position'].unique()

In [ ]:
rrh['finishing_position'] = rrh['finishing_position'].replace('PU','0')
rrh['finishing_position'] = rrh['finishing_position'].replace('DQ','0')
rrh['finishing_position'] = rrh['finishing_position'].replace('RR','0')
rrh['finishing_position'] = rrh['finishing_position'].replace('U','0')
rrh['finishing_position'] = rrh['finishing_position'].replace('F','0')
rrh['finishing_position'] = rrh['finishing_position'].replace('RO','0')
rrh['finishing_position'] = rrh['finishing_position'].replace('B','0')
rrh['finishing_position'] = rrh['finishing_position'].replace('R','0')
rrh['finishing_position'] = rrh['finishing_position'].replace('-','0')
rrh['finishing_position'] = rrh['finishing_position'].replace('UR','0')
rrh['finishing_position'] = rrh['finishing_position'].replace(0,'0')
rrh['finishing_position'] = rrh['finishing_position'].replace('S','0')
rrh['finishing_position'] = rrh['finishing_position'].replace('99','9')
rrh['finishing_position'] = rrh['finishing_position'].replace('DSQ ','0')
rrh['finishing_position'] = rrh['finishing_position'].replace('C','0')
#S
#S

In [ ]:
# Look at the unique values in the column 'finishing_position'
rrh['finishing_position'].unique()

In [ ]:
# Drop rows in column where value is not a number
df_horse = rrh[rrh['finishing_position'].isin([str(i) for i in range(0, 35)])]#.reset_index(drop=True)

In [ ]:
# View the data
df_horse.head(32)

In [ ]:
# Check the data types after removing special characters
df_horse.finishing_position.unique()

In [ ]:
df_horse

In [ ]:
# #  drop the rows where the “finish_position” is not a number (e.g. WV-A, WV). There may be accidents.
rrh = rrh[rrh.finishing_position.isin(['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30'])].reset_index() #,'31','32','33'
rrh.head()

In [ ]:
horse_id = df_horse['horse_id'].unique()
numHorse = len(horse_id)
horse_index = range(numHorse)

jockey = df_horse['jockey'].unique()
numJockey = len(jockey)
jockey_index = range(numJockey)

trainer = df_horse['trainer'].unique()
numTrainer = len(trainer)
trainer_index = range(numTrainer)

print(' Number of Horses:', numHorse, '\n', 'Number of Jockeys:', numJockey,'\n', 'Number of Trainers:', numTrainer)

In [ ]:
# Takes a while to run

# Add a column named recent_6_runs to the dataframe, which records the recent ranks of the horse in each entry. 
# The ranks are separated by “/”, and a record is like 1/2/6/3/4/7.

# Add a column named recent_ave_rank for each entry to the dataframe, 
# which records the average rank of the recent 6 runs of a horse
# function optimized to run on gpu 
@jit#(target_backend='cuda')
def func():
    df_horse['recent_6_runs'] = '0'
    df_horse['recent_ave_rank'] = '7'
    for i in range(len(df_horse['finishing_position'])):
          print(i)
          # temp will just give a list of all the finishing positions of that particular horse
          temp = df_horse[ : (i + 1)][df_horse.horse_id == df_horse.horse_id[i]][['finishing_position']]
          #print(temp)
          # we convert temp into a list, in reverse order, with the last race in front
          temp = temp['finishing_position'].values.tolist()[::-1]
          #print(temp)

          # we then take the first 6 elements of the list, and join them with a '/'
          df_horse['recent_6_runs'][i] = '/'.join(temp[:6])
          #print(df_horse['recent_6_runs'][i])

          # if list is not empty, then we convert string to float and take the average to get the recent_ave_rank
          if len(temp) != 0:
                temp_int = map(int,temp)
               
                temp_ave = np.mean(list(temp_int))
                print(temp_ave)
               
                df_horse['recent_ave_rank'][i] = temp_ave
func()
                

In [ ]:
# Check the data
df_horse.tail()

In [ ]:
# Add column of Distance

# The distance could be 1000, 1200, 1400, 1600, 1800, 2000, 2400, etc. 
# Some horses are good at short-distance races, while some are good at long-distance races.
@jit(target_backend='cuda')
def func1():
    df_horse['race_distance'] = '0'
    
    # Iterate through the dataframe
    for i in range(len(rrr['Race_ID'])):
        print(i)
        distance = rrr['Distance'][i]
        print(distance)
        df_horse['race_distance'][df_horse.race_id == rrr.Race_ID[i]] = distance
func1()

In [ ]:
# Add HorseWin, HorseRankTop3, HorseRankTop50Percent for use in Part3

# Sets a boolean mask (either 1 or 0) for each new column
df_horse['HorseWin'] = (df_horse.finishing_position == '1') + 0
df_horse['HorseRankTop3'] = (df_horse.finishing_position == '1') + (df_horse.finishing_position == '2') + (df_horse.finishing_position == '3') + 0

# Create a list of all the top1_indexes where the horse finished in the top
top1_index = df_horse.index[df_horse['finishing_position'] == '1'].tolist()

# Create another column where the horse finished in the top 50%
df_horse['HorseRankTop50Percent'] = '0'
for i in range(len(top1_index)-1):
    df_horse['HorseRankTop50Percent'][top1_index[i]:(top1_index[i] + int(round(0.5 * (top1_index[i + 1] - top1_index[i]))))] = '1'

# Created specifically for the last index because we cannot loop    
df_horse['HorseRankTop50Percent'][top1_index[len(top1_index) - 1]:(top1_index[len(top1_index) - 1] + 6)] = '1'


In [ ]:
df_horse.head(40)

In [ ]:
# Save the data
df_horse1 =df_horse.drop(['index'], axis=1)


In [ ]:
df_horse.to_csv('D:\\documentos\\IA Caballos\\hurdle_data\\df_horse.csv', index=False)

## Data Preparation

In [ ]:
import pandas as pd
import numpy as np
from numba import jit, cuda

df_horse = pd.read_csv('D:\\documentos\\IA Caballos\\hurdle_data\\df_horse.csv')
df_train = df_horse
#df_test = df_horse[(train_last_index + 1):(test_last_index + 1)]
df_unseen = df_horse
df_horse.head()


In [ ]:
df_unseen.head(2)

In [ ]:
@jit(target_backend='cuda')
def type_l():
    jockey = list(df_unseen['jockey'].unique())
    return jockey
print(type_l())

In [ ]:
@jit(target_backend='cuda')
def type_2():
    trainer = list(df_unseen['trainer'].unique())
    return trainer
print(type_2())

In [ ]:
# Calculate jockey_ave_rank, trainer_ave_rank ONLY for training and test data
# jockey_ave_rank: records the average rank of the jockey
# trainer_ave_rank: records the average rank of the trainer
@jit(target_backend='cuda')
def cal_ave_rank(df):
    jockey = type_l()
#jockey.head()
    print(len(jockey))
    df['jockey_ave_rank'] = '7'
    
    for i in range(len(jockey)):
        print(i)
        temp = df[df.jockey == jockey[i]][['finishing_position']]
        temp = temp['finishing_position'].values.tolist()
        if len(temp) != 0:
            temp_int = map(int, temp)
            temp_ave = np.mean(list(temp_int))
            print('temp_ave')
            print(temp_ave)
            df['jockey_ave_rank'][df.jockey == jockey[i]] = temp_ave


    df['trainer_ave_rank'] = '7'
    trainer = type_2()
    for i in range(len(trainer)):
        print(i)
        temp = df[df.trainer == trainer[i]][['finishing_position']]
        temp = temp['finishing_position'].values.tolist()
        if len(temp) != 0:
            temp_int = map(int,temp)
            temp_ave = np.mean(list(temp_int))
            df['trainer_ave_rank'][df.trainer == trainer[i]] = temp_ave

    return df


In [ ]:
df_train = cal_ave_rank(df_train)

In [ ]:
# Save the train set
df_train.to_csv('D:\\documentos\\IA Caballos\\hurdle_data\\df_test.csv', index=False)

# Save the test set

df_train.to_csv('D:\\documentos\\IA Caballos\\hurdle_data\\df_train.csv', index=False)


In [ ]:
# Port over the jockey_ave_rank and trainer_ave_rank to the unseen set
@jit(target_backend='cuda')
def unseen():
    df_unseen['jockey_ave_rank'] = '7'
    df_unseen_jockey =  type_l()
    for i in range(len(df_unseen_jockey)):
        if df_unseen_jockey[i] in df_train['jockey'].unique():
            df_unseen['jockey_ave_rank'][df_unseen.jockey == df_unseen_jockey[i]]\
                = df_train[df_train.jockey == df_unseen_jockey[i]]['jockey_ave_rank'].tolist()[0]

    df_unseen['trainer_ave_rank'] = '7'
    df_unseen_trainer=  type_2()
    for i in range(len(df_unseen_trainer)):
        if df_unseen_trainer[i] in df_train['trainer'].unique():
            df_unseen['trainer_ave_rank'][df_unseen.trainer == df_unseen_trainer[i]]\
                = df_train[df_train.trainer == df_unseen_trainer[i]]['trainer_ave_rank'].tolist()[0]


In [ ]:
unseen()

In [ ]:
df_unseen.to_csv('D:\\documentos\\IA Caballos\\hurdle_data\\df_unseen.csv', index=False)